Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [9]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [10]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [11]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [12]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [22]:
batch_size = 128
beta = 0.01


graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    hidden_layer_size = 1024

    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    hidden_biases = tf.Variable(tf.zeros([hidden_layer_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    output_bias = tf.Variable(tf.zeros([num_labels]))


    def myNetwork(input_data):
        # Hidden layer
        activations_hidden = tf.matmul(input_data, hidden_weights) + hidden_biases
        hidden_y = tf.nn.relu(activations_hidden)

        # Output layer
        output_activations = tf.matmul(hidden_y, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
       
        return output

    # Training computation.
    output_prediction = myNetwork(tf_train_dataset)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_prediction)
    regularizer = tf.nn.l2_loss (hidden_weights) + tf.nn.l2_loss (output_weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.

    valid_prediction = myNetwork(tf_valid_dataset)
    test_prediction = myNetwork(tf_test_dataset)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, output_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3143.967773
Minibatch accuracy: 7.0%
Validation accuracy: 13.1%
Minibatch loss at step 500: 22.776562
Minibatch accuracy: 62.5%
Validation accuracy: 66.0%
Minibatch loss at step 1000: 1.986031
Minibatch accuracy: 76.6%
Validation accuracy: 73.8%
Minibatch loss at step 1500: 1.862716
Minibatch accuracy: 72.7%
Validation accuracy: 74.3%
Minibatch loss at step 2000: 1.888084
Minibatch accuracy: 72.7%
Validation accuracy: 73.9%
Minibatch loss at step 2500: 1.849067
Minibatch accuracy: 72.7%
Validation accuracy: 74.0%
Minibatch loss at step 3000: 1.916909
Minibatch accuracy: 71.9%
Validation accuracy: 73.7%
Test accuracy: 79.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % 2
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, output_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3150.384521
Minibatch accuracy: 3.9%
Validation accuracy: 7.4%
Minibatch loss at step 500: 22.487272
Minibatch accuracy: 97.7%
Validation accuracy: 66.9%
Minibatch loss at step 1000: 1.798375
Minibatch accuracy: 99.2%
Validation accuracy: 69.7%
Minibatch loss at step 1500: 1.649502
Minibatch accuracy: 100.0%
Validation accuracy: 70.4%
Minibatch loss at step 2000: 1.644829
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 2500: 1.642590
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 3000: 1.641074
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Test accuracy: 77.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [60]:
batch_size = 128
beta = 0.01


graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    hidden_layer_size = 1024

    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    hidden_biases = tf.Variable(tf.zeros([hidden_layer_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    output_bias = tf.Variable(tf.zeros([num_labels]))
    rate = tf.placeholder("float")

    def myNetwork_t(input_data):
        # Hidden layer
        activations_hidden = tf.matmul(input_data, hidden_weights) + hidden_biases
        hidden_y = tf.nn.relu(activations_hidden)
        hidden_y = tf.nn.dropout(hidden_y, rate)
        
        # Output layer
        output_activations = tf.matmul(hidden_y, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
        
        return output

    def myNetwork(input_data):
        # Hidden layer
        activations_hidden = tf.matmul(input_data, hidden_weights) + hidden_biases
        hidden_y = tf.nn.relu(activations_hidden)
        
        # Output layer
        output_activations = tf.matmul(hidden_y, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
        
        return output
    
    # Training computation.
    output_prediction = myNetwork_t(tf_train_dataset)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_prediction)
    regularizer = tf.nn.l2_loss (hidden_weights) + tf.nn.l2_loss (output_weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.

    valid_prediction = myNetwork(tf_valid_dataset)
    test_prediction = myNetwork(tf_test_dataset)

In [61]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % 2
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, rate : 0.5}
        _, l, predictions = session.run([optimizer, loss, output_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3146.407715
Minibatch accuracy: 10.9%
Validation accuracy: 9.0%
Minibatch loss at step 500: 22.487724
Minibatch accuracy: 93.8%
Validation accuracy: 63.5%
Minibatch loss at step 1000: 1.802971
Minibatch accuracy: 99.2%
Validation accuracy: 70.5%
Minibatch loss at step 1500: 1.657976
Minibatch accuracy: 99.2%
Validation accuracy: 71.2%
Minibatch loss at step 2000: 1.653181
Minibatch accuracy: 99.2%
Validation accuracy: 71.6%
Minibatch loss at step 2500: 1.649063
Minibatch accuracy: 99.2%
Validation accuracy: 71.3%
Minibatch loss at step 3000: 1.652316
Minibatch accuracy: 99.2%
Validation accuracy: 71.1%
Test accuracy: 77.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
